In [ ]:
import os
import cv2

In [ ]:
def get_data(dir_path="/content/drive/MyDrive/2기 딥러닝 프로젝트/train/"):
    labels = ['Closed', 'Open']
    IMG_SIZE = 145
    data = []
    for label in labels:
        path = os.path.join(dir_path, label)
        class_num = labels.index(label)
        print(class_num)
        for img in os.listdir(path): # closed 폴더 안에 _1.jpg, open 폴더 안에 1.jpg 
            try:
                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_COLOR) #/content/drive/MyDrive/2기 딥러닝 프로젝트/train/Closed/_0.jpg
                resized_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                data.append([resized_array, class_num])
            except Exception as e:
                print(e)
    return data

In [ ]:
data_train = get_data()

0
1
OpenCV(4.6.0) /io/opencv/modules/imgproc/src/resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'resize'



In [ ]:
X = []
y = []
for feature, label in data_train:
    X.append(feature)
    y.append(label)

In [ ]:
import numpy as np
X = np.array(X)
y = np.array(y)

In [ ]:
from sklearn.preprocessing import LabelBinarizer
label_bin = LabelBinarizer()
y = label_bin.fit_transform(y)

In [ ]:
from sklearn.model_selection import train_test_split
seed = 42
test_size = 0.30
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=seed, test_size=test_size)

## **train**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
import torch.optim as optim

train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomRotation(10),
    transforms.RandomHorizontalFlip(),
])

class eyes_dataset(Dataset):
    def __init__(self, x_file_paths, y_file_path, transform=None):
        self.x_files = x_file_paths
        self.y_files = y_file_path
        self.transform = transform

    def __getitem__(self, idx):
        x = self.x_files[idx]
        x = torch.from_numpy(x).float()

        y = self.y_files[idx]
        y = torch.from_numpy(y).float()

        return x, y

    def __len__(self):
        return len(self.x_files)

train_dataset = eyes_dataset(X_train, y_train, transform=train_transform)

In [ ]:
X_train.shape

(1015, 145, 145, 3)

In [ ]:
y_train.shape

(1015, 1)

In [ ]:
#train_generator = ImageDataGenerator(rescale=1/255, zoom_range=0.2, horizontal_flip=True, rotation_range=30)
#test_generator = ImageDataGenerator(rescale=1/255)

#train_generator = train_generator.flow(np.array(X_train), y_train, shuffle=False)
#test_generator = test_generator.flow(np.array(X_test), y_test, shuffle=False)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(1536, 512) #k는 conv3 하고 나온 이미지의 shape를 일자로 펴준거기 때문에 아직 계산을 안 해서 이미지가 다 모아지면 계산하면 된다
        self.fc2 = nn.Linear(512, 1)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)),2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = F.max_pool2d(F.relu(self.conv3(x)), 2)
        x = x.reshape(-1, 1536)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)


        return x

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
import torch.optim as optim

In [ ]:
model = Net()
model.to('cuda')

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

epochs = 50

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
len(train_dataloader)

32

In [ ]:
X_train = transforms.ToTensor().fit_transform(X_train)

AttributeError: ignored

In [ ]:
a = train_dataset[0]
len(a)
#print('a.shape' ,a.shape)
print('X_train.shape', X_train.shape)
b = train_dataset[1]
#print('b.shape', b.shape)
print('y_train.shape', y_train.shape)

X_train.shape (1015, 145, 145, 3)
y_train.shape (1015, 1)


In [ ]:
x_train.

In [ ]:
for epoch in range(epochs):
    running_loss = 0.0
    running_acc = 0.0

    model.train()

    X_train = X_train.to('cuda')
    y_train = y_train.to('cuda')
    input_1 = X_train.transpose(1, 3).transpose(2, 3)
    for i, data in enumerate(train_dataloader, 0):

        optimizer.zero_grad()

        outputs = model(input)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        running_acc += accuracy(outputs, labels)

        if i % 80 == 79:
            print('epoch: [%d/%d] train_loss: %.5f train_acc: %.5f' % (
                epoch + 1, epochs, running_loss / 80, running_acc / 80))
            running_loss = 0.0

print("learning finish")
torch.save(model.state_dict(), PATH)

AttributeError: ignored

In [ ]:
history = model.fit(train_generator, epochs=50, validation_data=test_generator, shuffle=True, validation_steps=len(test_generator))

Epoch 1/50
32/32 [==============================] - 19s 239ms/step - loss: 654.8847 - accuracy: 0.2443 - val_loss: 2267.6921 - val_accuracy: 0.1284
Epoch 2/50
32/32 [==============================] - 6s 186ms/step - loss: 71709.4453 - accuracy: 0.2719 - val_loss: 157213.4062 - val_accuracy: 0.0023
Epoch 3/50
32/32 [==============================] - 6s 187ms/step - loss: 1262332.2500 - accuracy: 0.2522 - val_loss: 1948590.3750 - val_accuracy: 0.9083
Epoch 4/50
32/32 [==============================] - 6s 186ms/step - loss: 14875299.0000 - accuracy: 0.2443 - val_loss: 10188523.0000 - val_accuracy: 0.8050
Epoch 5/50
32/32 [==============================] - 6s 185ms/step - loss: 70441072.0000 - accuracy: 0.2749 - val_loss: 106571240.0000 - val_accuracy: 0.0000e+00
Epoch 6/50
32/32 [==============================] - 6s 187ms/step - loss: 229548912.0000 - accuracy: 0.2355 - val_loss: 286503296.0000 - val_accuracy: 0.0000e+00
Epoch 7/50
32/32 [==============================] - 6s 188ms/step - 

KeyboardInterrupt: ignored

# **모델 새로 쌓기**

In [ ]:
# set parameters
learning_rate = 0.0004
num_epoch = 10


# CNN 모델학습
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Conv2d(in_channels = 3,out_channels =16,kernel_size = 5),   # [4,3,32,32]->[16,3,28,28]
            nn.ReLU(),
            nn.MaxPool2d(2,2),                                             # [4,16,28,28]->[4,16,14,14]
            nn.Conv2d(in_channels = 16,out_channels =32,kernel_size = 5),  # [4,16,14,14]->[4,32,10,10]
            nn.ReLU(),
            nn.MaxPool2d(2,2),                                             # [4,32,10,10]->[4,32,5,5]
            nn.Conv2d(32,64,5,padding=2),                                  # [4,32,5,5]->[4,64,5,5]
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(64 * 5 * 5, 120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, 10)
        )

    def forward(self, x):
      out = self.layer(x)
      return out

net = Net()

# 손실함수와 옵티마이저 정하기
import torch.optim as optim
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
X_train.shape

torch.Size([1015, 3, 145, 145])

In [ ]:
X_train = X_train.reshape(1015,3,145,145)

In [ ]:
y_train.shape

(1015, 1)

In [ ]:
train = torch.tensor([1015,145,145,3,1015])
X_train = torch.tensor(X_train)
y_train = torch.tensor(y_train)
train[0] = X_train
train[1] = y_train

RuntimeError: ignored

In [ ]:
#신경망 학습하기
for epoch in range(2):   # 데이터셋을 수차례 반복합니다.
    running_loss = 0.0
    for i, data in enumerate(X_train, 0):
        # [inputs, labels]의 목록인 data로부터 입력을 받은 후;
        inputs = data
        labels = ['Closed', 'Open']

        # 변화도(Gradient) 매개변수를 0으로 만들고
        optimizer.zero_grad()

        # 순전파 + 역전파 + 최적화를 한 후
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # 통계를 출력합니다.
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

RuntimeError: ignored